In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.util import ngrams
import plotly.express as px
import scipy.stats
import math
import statistics
import string 
import re
from collections import defaultdict
from gensim import corpora, models
from nltk.text import Text
# ----------------------------------------
import random
from nltk.tokenize import word_tokenize, sent_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# ----------------------------------------
print('Success!')

Success!


8 data (x) 
ranking negative(1-3) positive(4,5) (x) 
single words + Bi-grame + Tri-gram

### Import all the data 

In [2]:
# data importing 
lowes_6026 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3286026.csv')
lowes_0314 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3380314.csv')
lowes_0511 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_50280511.csv') 
lowes_6162 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_1003066162.csv')
lowes_6209 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5000026209.csv')
lowes_9855 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5001899855.csv')

In [3]:
# data importing 
homedepot_4796 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_204074796.csv')
homedepot_7071 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_319247071.csv')

In [4]:
# set display option to max to see all information in review column
pd.set_option('display.max_colwidth', -1)

C:\Users\lxf12\AppData\Local\Temp\ipykernel_24240\779606820.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [5]:
# lowes_0314.head(2)

In [6]:
# drop null value
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]
namelist = ['lowes_6026', 'lowes_0314', 'lowes_0511', 'lowes_6162',
                    'lowes_6209', 'lowes_9855', 
                     'homedepot_4796', 'homedepot_7071']
n = 0 

for i in datalist: 
    # drop rows with Not a Number (NaN) and None values
    name = namelist[n]
    n += 1
    print(name + ' has null rows amount: '+ str(i['Review'].isna().sum()))
    i.dropna(subset=['Review'], inplace=True)
    print('after cleaning, ' + name + ' has null rows amount: ' 
          + str(i['Review'].isna().sum()))
    

lowes_6026 has null rows amount: 30
after cleaning, lowes_6026 has null rows amount: 0
lowes_0314 has null rows amount: 10
after cleaning, lowes_0314 has null rows amount: 0
lowes_0511 has null rows amount: 15
after cleaning, lowes_0511 has null rows amount: 0
lowes_6162 has null rows amount: 297
after cleaning, lowes_6162 has null rows amount: 0
lowes_6209 has null rows amount: 76
after cleaning, lowes_6209 has null rows amount: 0
lowes_9855 has null rows amount: 170
after cleaning, lowes_9855 has null rows amount: 0
homedepot_4796 has null rows amount: 186
after cleaning, homedepot_4796 has null rows amount: 0
homedepot_7071 has null rows amount: 9
after cleaning, homedepot_7071 has null rows amount: 0


In [7]:
# lowes_9855.info()

In [8]:
# datalist[0].isna().sum()

### Separate Negative and Positive Ranking

In [9]:
# 8 data x 2 categories 
try:
    user_select = int(input("Please select negative(0) or positive ranking(1): "))
except: 
    print("Your input is not correct. Please input again according the tips.")

if user_select == 0:
    # get reviews that have given 1, 2, and 3 stars (negative)
    lowes_6026 = lowes_6026.loc[lowes_6026['Stars'] <= 3]
    lowes_0314 = lowes_0314.loc[lowes_0314['Stars'] <= 3]
    lowes_0511 = lowes_0511.loc[lowes_0511['Stars'] <= 3]
    lowes_6162 = lowes_6162.loc[lowes_6162['Stars'] <= 3]
    lowes_6209 = lowes_6209.loc[lowes_6209['Stars'] <= 3]
    lowes_9855 = lowes_9855.loc[lowes_9855['Stars'] <= 3]
    # get reviews that have given 1, 2, and 3 stars (negative)
    homedepot_4796 = homedepot_4796.loc[homedepot_4796['Stars'] <= 3]
    homedepot_7071 = homedepot_7071.loc[homedepot_7071['Stars'] <= 3]
elif user_select == 1:
    # get reviews that have given 4 and 5 stars (positive)
    lowes_6026 = lowes_6026.loc[lowes_6026['Stars'] > 3]
    lowes_0314 = lowes_0314.loc[lowes_0314['Stars'] > 3]
    lowes_0511 = lowes_0511.loc[lowes_0511['Stars'] > 3]
    lowes_6162 = lowes_6162.loc[lowes_6162['Stars'] > 3]
    lowes_6209 = lowes_6209.loc[lowes_6209['Stars'] > 3]
    lowes_9855 = lowes_9855.loc[lowes_9855['Stars'] > 3]
    # get reviews that have given 4 and 5 stars (positive)
    homedepot_4796 = homedepot_4796.loc[homedepot_4796['Stars'] > 3]
    homedepot_7071 = homedepot_7071.loc[homedepot_7071['Stars'] > 3]
else: 
    print("Your input is not correct. Please input again according the tips.")

Please select negative(0) or positive ranking(1): 0


In [10]:
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

In [11]:
# lowes_0314.info()

### Lower case for all words & Remove punctuation

In [12]:
# download package punkt for word_tokenize
# nltk.download('punkt')
# download package for stopwords
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

### Store them to text list

In [13]:
# before the cleaning 
text_01 = lowes_6026['Review'].to_string(index=False)
# text_01

In [14]:
# Create a translation table with all punctuation characters mapped to None
translator = str.maketrans('', '', string.punctuation)

text = [0] * 8
n = 0 

for i in datalist:
    # convert pandas series data to string
    text[n] = i['Review'].to_string(index=False)
    # convert all characters to lowercase
    text[n] = text[n].lower()
    # remove punctuations using RegexpTokenizer
    text[n] = text[n].translate(translator)
    # Use regex to remove all numbers
    text[n] = re.sub(r'\d+', '', text[n])
    n += 1

In [15]:
# the result has been single words 
# text[0]

In [16]:
type(text[0])

str

In [17]:
# renew the datalist --> can't be deleted 
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

### Stop words

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    # customize stop_words by including a custom list of words
    custom_stop_words = ['toilet']
#     custom_stop_words = ['new','toilet', 'factory', 'flush', 'sku', 'photos', 'stay', 
#                                      'lid', 'good', 'great', 'handle', 'top', 'high', 'urine']
    stop_words.update(custom_stop_words)
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lxf12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
for i in range(8):
    text[i] = remove_stopwords(text[i])

In [20]:
# renew the datalist --> can't be deleted 
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

### Text Tokenization

In [21]:
# remove punctuations using RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tokens = [0] * 8
for i in range(8):
    tokens[i] = tokenizer.tokenize(text[i])

In [22]:
# tokens[0]

### Bi-gram and Tri-gram

In [56]:
# N-gram generation
N = int(input('How many words do you want to aggregate? (1-8)'))

trigrams = [0] * 8
for i in range(8):
    trigrams[i] = list(ngrams(tokens[i], N))

How many words do you want to aggregate? (1-8)4


In [24]:
# connect the three words together 
words = [0] * 8
for i in range(8):
    words[i] = [' '.join(tr) for tr in trigrams[i]]

### Frequency Analysis

In [57]:
# select the data 
n = int(input('Which one do you want to look at? (1-8)'))

word_frequency = FreqDist(trigrams[n-1])

# Identify co-occurring trigrams
cooccurring_trigrams = []
for trigram in word_frequency:
    if word_frequency[trigram] > 1:
        cooccurring_trigrams.append(trigram)

word_frequency.most_common(10)

# word_frequency.plot(30,cumulative=False)
# plt.show()

Which one do you want to look at? (1-8)1


[(('american', 'standard', 'customer', 'service'), 3),
 (('needs', 'cleaned', 'every', 'day'), 2),
 (('american', 'standard', 'mainstream', 'model'), 2),
 (('one', 'market', 'flow', 'master'), 2),
 (('small', 'amount', 'water', 'enters'), 2),
 (('amount', 'water', 'enters', 'bowl'), 2),
 (('water', 'enters', 'bowl', 'enough'), 2),
 (('like', 'facts', 'uses', 'little'), 1),
 (('facts', 'uses', 'little', 'water'), 1),
 (('uses', 'little', 'water', 'chair'), 1)]

### Topic Modeling
#### Latent Dirichlet Allocation (LDA), which is a popular unsupervised learning algorithm used for topic modeling.

In [26]:
from gensim import corpora, models

In [27]:
# Step 1: select the data 
n = int(input('Which one do you want to look at? (1-8)'))

# Step 2: Create a dictionary of words and convert the text into a bag-of-words format
dictionary = corpora.Dictionary([words[n-1]])
corpus = [dictionary.doc2bow([word]) for word in words[n-1]]

# Step 3: Apply LDA to the corpus to identify topics in the text
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)

# Step 4: Print the topics and the top words associated with each topic
for i, topic in lda_model.show_topics(formatted=True, num_topics=3, num_words=4):
    print("Topic {}: {}".format(i, topic))

Which one do you want to look at? (1-8)1
Topic 0: 0.001*"i m happy" + 0.001*"hair line crack" + 0.001*"water keeps running" + 0.001*"flush doesnt clean"
Topic 1: 0.001*"flush valve assembly" + 0.001*"cheap plastic seat" + 0.001*"bought two toilets" + 0.001*"adjust water level"
Topic 2: 0.001*"six eight times" + 0.001*"flushed several times" + 0.001*"nasty rusty bolts" + 0.001*"small rubber washer"


### Text Summarization
#### In summary, while topic modeling aims to uncover the underlying themes and topics in a large corpus of text, text summarization aims to provide a condensed version of a longer text, capturing the essential information and key points.
#### Two different approaches are used for Text Summarization
1. Extractive Summarization: In Extractive Summarization, we identify essential phrases or sentences from the original text and extract only these phrases from the text. These extracted sentences would be the summary.
2. Abstractive Summarization: We work on generating new sentences from the original text in the Abstractive Summarization approach. The abstractive method contrasts the approach described above, and the sentences generated through this approach might not even be present in the original text.

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [29]:
# select the data 
n = int(input('Which one do you want to look at? (1-8)'))
result_text_1 = datalist[n-1]['Review'].to_string(index=False)

Which one do you want to look at? (1-8)1


In [30]:
stopWords = set(stopwords.words("english"))
words = word_tokenize(result_text_1)
freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

In [31]:
sentences = sent_tokenize(result_text_1)
sentenceValue = dict()
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq

In [32]:
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

In [33]:
average = int(sumValues / len(sentenceValue))

In [34]:
# Storing sentences into our summary.
summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.8 * average)):
        summary += " " + sentence

In [35]:
print(summary)

 After removing the old toilet and setting the base and tank I opened the new box only to find the tank lid cracked, this is poorly put together and hard to get it swallow anything                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [36]:
# import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
# from string import punctuation
# from heapq import nlargest
# from spacy.lang.en.examples import sentences 

In [37]:
# def summarize(text, per):
#     nlp = spacy.load('en_core_web_sm')
#     doc= nlp(text)
#     tokens=[token.text for token in doc]
#     word_frequencies={}
#     for word in doc:
#         if word.text.lower() not in list(STOP_WORDS):
#             if word.text.lower() not in punctuation:
#                 if word.text not in word_frequencies.keys():
#                     word_frequencies[word.text] = 1
#                 else:
#                     word_frequencies[word.text] += 1
#     max_frequency=max(word_frequencies.values())
#     for word in word_frequencies.keys():
#         word_frequencies[word]=word_frequencies[word]/max_frequency
#     sentence_tokens= [sent for sent in doc.sents]
#     sentence_scores = {}
#     for sent in sentence_tokens:
#         for word in sent:
#             if word.text.lower() in word_frequencies.keys():
#                 if sent not in sentence_scores.keys():                            
#                     sentence_scores[sent]=word_frequencies[word.text.lower()]
#                 else:
#                     sentence_scores[sent]+=word_frequencies[word.text.lower()]
#     select_length=int(len(sentence_tokens)*per)
#     summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
#     final_summary=[word.text for word in summary]
#     summary=''.join(final_summary)
#     return summary

In [38]:
# datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
#                     lowes_6209, lowes_9855, 
#                      homedepot_4796, homedepot_7071]

In [39]:
# text_6026 = lowes_6026['Review'].to_string(index=False)

In [40]:
# print(summarize(result_text_1, 0.03))
# # per is the percentage (0 to 1) of sentences you want to extract

### Context Analysis
#### Check out the context of words having highest frequency

In [41]:
# select the data 
n = int(input('Which one do you want to look at? (1-8)'))
result_text_1 = text[n-1]

Which one do you want to look at? (1-8)1


In [42]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(result_text_1)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "j":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of ADJ: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of ADJ: 1409



In [43]:
# see each Adjective frequence
ADJ_f_1 = FreqDist(result_adj_1)
# ADJ_f_1
print(f"The top 10 most common Adjective. word appear in review is: {ADJ_f_1.most_common(10)}")

The top 10 most common Adjective. word appear in review is: [('flush', 70), ('american', 36), ('new', 33), ('good', 32), ('old', 23), ('small', 19), ('great', 18), ('little', 17), ('much', 15), ('bad', 15)]


In [44]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(result_text_1)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "n":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of noun: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of noun: 2804



In [45]:
# see each noun frequence
noun_f_1 = FreqDist(result_adj_1)
print(f"The top 10 most common Noun. word appear in review is: {noun_f_1.most_common(10)}")

The top 10 most common Noun. word appear in review is: [('water', 140), ('tank', 62), ('seat', 56), ('bowl', 49), ('flush', 47), ('toilets', 40), ('standard', 39), ('time', 32), ('parts', 26), ('level', 26)]


In [46]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(result_text_1)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "v":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of verb: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of verb: 1566



In [47]:
# see each noun frequence
noun_f_1 = FreqDist(result_adj_1)
print(f"The top 10 most common Verb. word appear in review is: {noun_f_1.most_common(10)}")

The top 10 most common Verb. word appear in review is: [('installed', 42), ('replace', 32), ('purchased', 32), ('bought', 29), ('buy', 27), ('get', 26), ('flushing', 25), ('running', 23), ('said', 22), ('went', 19)]


#### concordance function: to locate the key word and get the context
https://www.nltk.org/howto/concordance.html

In [48]:
# find word in the text
# select the data 
# n = int(input('Which one do you want to look at? (1-8)'))
# textList_1 = Text(datalist[n-1]['Review'].to_string(index=False))
textList_1 = Text(tokens[n-1])
# textList_1.concordance(['flush'], lines=150, width=150)
# textList_1.concordance(['american', 'standard'], lines=10, width=300)

word = 'water'
# Find the concordance of a specific word (e.g. "Caesar") in the text
concordance_list = textList_1.concordance_list([word], lines=150, width=150)

# Sample 20 lines from the concordance list
sampled_lines = random.sample(concordance_list, 10)

# Print the sampled lines
print('Here is the samples of the ' + word + ":")
for line in sampled_lines:
    print(line.line)

Here is the samples of the water:
nt fine installed became problem old years plus base went rather easily water tank became problem would seat properly developed leaks wabbled afraid 
l guess trouble flushing one time far better old one course still brush water ring like old one wife likes taller seat height still works well ok uni
cannot begin express years discomfort disagreements family endured whim water sense thats deceptive code low flow ive quietly tried defend purchase t
ank boltgaskets next time buy cadet home depot installed per directions water supply would shut continued pouring water bowl overflow tube flow valve
ng number go use master bathroom bought april seating area rather small water flush rear bowl consequently someone sits far back seat flush wont remo
control inspection factory purchased months ago months age kept hearing water running one night discovered running water tank set position running ov
tion occurrence cracked purchased two toilets fine one constant ru

#### New Method to Get Context of Key Words
using original text of reviews and locate the full sentence, then sample them to read

In [49]:
import re
import nltk
import random

nltk.download('punkt')  # Download the Punkt tokenizer

result_text_1 = datalist[n-1]['Review'].to_string(index=False)
# result_text_1 = text[n-1]
keyword = "clean"
max_sentence_length = 150
num_samples = 10

sentences = nltk.sent_tokenize(result_text_1)
matching_sentences = []

for sentence in sentences:
    if keyword in sentence:
        if len(sentence) <= max_sentence_length:
            matching_sentences.append(sentence)

if len(matching_sentences) >= num_samples:
    samples = random.sample(matching_sentences, num_samples)
    for sample in samples:
        print(sample)
else:
    print("Not enough matching sentences.")

We bought the Tropic 1pc skirted toilet for ease of cleaning & it surely does not disappoint in that arena.
The other issue is the staining below the water line, it seems the porcelain glaze is very rough and holds stains not cleaning well at all.
We are having to constantly clean the bowl because any of the waste put into it tends to stick.
It does not self clean.
So you get a fairly strong flush, but the bowl doesn't get rinsed clean.
Would be a nightmare to clean since the underside is an open grid.
Well, waste still sticks to the bowl since it doesn't hold much water and the flush isn't enough to really clean the bowl.
Apart from that, this model doesn't clean itself really well.
For the first year, cleaning was almost effortless.
They need frequent cleaning.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lxf12\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


locate the fixed length context, then sample them to read

In [50]:
import re
import random

result_text_1 = datalist[n-1]['Review'].to_string(index=False)
# result_text_1 = text[n-1]
keyword = "clean"
context_length = 100
num_samples = 10

pattern = re.compile(re.escape(keyword))

matches = [(match.start(), match.end()) for match in pattern.finditer(result_text_1)]

matching_contexts = []
for start, end in matches:
    context_start = max(0, start - context_length)
    context_end = min(len(result_text_1), end + context_length)
    context = result_text_1[context_start:end] + result_text_1[end:context_end]
    matching_contexts.append(context)

if len(matching_contexts) >= num_samples:
    samples = random.sample(matching_contexts, num_samples)
    for sample in samples:
        print(sample)
else:
    print("Not enough matching contexts.")


toilet, I does not remove paper, or waste from the FRONT of the toilet bold when flushed! I have to clean the toilet almost every time the toilet is used! The flush valve provided with this unit hangs up a
eaves bits on the sides of the stool. In the 2 weeks we had it, we found that we had to continually clean the stool to keep it looking decent. In the end, we are obviously very disappointed.               
                                                             
about half the time the flush doesn't clean bowl,i would hate to have guest use.                                                               
       
We have had this unit in the main bathroom for about 1.5 years. We are having to constantly clean the bowl because any of the waste put into it tends to stick. It seems that the bowl was sand cast 
                                               
We bought the Tropic 1pc skirted toilet for ease of cleaning & it surely does not disappoint in that arena. It is also low on wa

#### Text Summarization on the Matching sentences 

In [51]:
matching_sentences = ' '.join(matching_sentences)

In [52]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [53]:
sentences = sent_tokenize(matching_sentences)
sentenceValue = dict()
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
                
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]
    
average = int(sumValues / len(sentenceValue))

In [54]:
# Storing sentences into our summary.
summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
        summary += " " + sentence
print(summary)

 Well, waste still sticks to the bowl since it doesn't hold much water and the flush isn't enough to really clean the bowl. They also state the toilet is only to remove solids, NOT clean the bowl. The other issue is the staining below the water line, it seems the porcelain glaze is very rough and holds stains not cleaning well at all. So you get a fairly strong flush, but the bowl doesn't get rinsed clean. Sometimes you need to flush a few times or use a toilet brush often, to get the bowl clean.


#### New Methond on Text Summarization
Compute a score for each sentence based on its importance to the overall meaning of the text. The score can be based on various criteria, such as word frequency, sentence length, or semantic similarity to other sentences in the text.

In [55]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# text = "The quick brown fox jumps over the lazy dog. The quick brown dog jumps over the lazy cat. The quick brown cat jumps over the lazy mouse. The quick brown mouse jumps over the lazy rabbit. The quick brown rabbit jumps over the lazy deer."

parser = PlaintextParser.from_string(matching_sentences, Tokenizer("english"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, sentences_count=8)

for sentence in summary:
    print(sentence)

Well, waste still sticks to the bowl since it doesn't hold much water and the flush isn't enough to really clean the bowl.
I have to clean the toilet almost every time the toilet is used!
The small amount of water that enters the bowl is not enough to really flush the walls of the bowl and get them clean.
Sometimes you need to flush a few times or use a toilet brush often, to get the bowl clean.
They need frequent cleaning.
The flush doesn't clean bowl well.
Doesn't clean well.
We bought the Tropic 1pc skirted toilet for ease of cleaning & it surely does not disappoint in that arena.
